In [2]:
import numpy as np
import pandas as pd


In [162]:
DIRECTIONS = {
    "r": (1, 0),
    "rb": (1, 1),
    "b": (0, 1),
    "bl": (-1, 1),
    "l": (-1, 0),
    "lt": (-1, -1),
    "t": (0, -1),
    "tr": (1, -1),
}

def move(start, direction):
    return [x + y for x, y in zip(start, direction)]

# move(start_point, DIRECTIONS["rb"])
def get_direction(start, end):
    
    start_x, start_y = start
    end_x, end_y = end
    
    if end_x > start_x and end_y > start_y:
        directions = [DIRECTIONS["r"], DIRECTIONS["rb"], DIRECTIONS["b"]]
    elif end_x > start_x and end_y == start_y:
        directions = [DIRECTIONS["tr"], DIRECTIONS["r"], DIRECTIONS["rb"]]
    elif end_x > start_x and end_y < start_y:
        directions = [DIRECTIONS["t"], DIRECTIONS["tr"], DIRECTIONS["r"]]
    elif end_x == start_x and end_y < start_y:
        directions = [DIRECTIONS["lt"], DIRECTIONS["t"], DIRECTIONS["tr"]]
    elif end_x < start_x and end_y < start_y:
        directions = [DIRECTIONS["l"], DIRECTIONS["lt"], DIRECTIONS["t"]]
    elif end_x < start_x and end_y == start_y:
        directions = [DIRECTIONS["bl"], DIRECTIONS["l"], DIRECTIONS["lt"]]
    elif end_x < start_x and end_y > start_y:
        directions = [DIRECTIONS["b"], DIRECTIONS["bl"], DIRECTIONS["l"]]
    elif end_x == start_x and end_y > start_y:
        directions = [DIRECTIONS["rb"], DIRECTIONS["b"], DIRECTIONS["bl"]]
        
    return [move(start, direct) for direct in directions]

def __serialize(cords):
    return ",".join([str(x) for x in cords])

def __deserialize(cords_str):
    return tuple(cords_str.split(","))

def __check_valid(cords):
    return True

def add_visited(cords, visited=set()):
    cords_str = __serialize(cords)
    visited.add(cords_str)
    return visited



def filter_pos_by_minvalue(feats_array, cords):
    feats_selected = np.array([feats_array[x,y] for x, y in cords])
    # feats_selected = np.array([1,1,1])
    mask = (feats_selected == feats_selected.min())

    if mask.sum() == 1:
        next_idx = [feats_selected.argmin()]
    else:
        next_idx = [idx for idx, val in enumerate(mask) if val]
    
    return [cords[idx] for idx in next_idx]


def filter_pos_by_mindist(end, cords):
    
    rst = []
    for vec in cords:
        dist = sum([(x-y)**2 for x,y in zip(vec, end)])
        rst += [dist]
    rst = np.array(rst)
        
    mask = (rst == rst.min())

    if mask.sum() == 1:
        next_idx = [rst.argmin()]
    else:
        next_idx = [idx for idx, val in enumerate(mask) if val]
        
    return [cords[idx] for idx in next_idx]

def process_one_round(current_pos, end, visited):
    next_pos = get_direction(current_pos, end)
    next_pos = [x for x in next_pos if __serialize(x) not in visited]
    next_pos = [x for x in next_pos if __check_valid(x)]
    
    for cord in next_pos:
        visited = add_visited(cord, visited=visited)
        
    if end in next_pos:
        return [end], visited
    else:
        # print("next position:", next_pos)
        if len(next_pos)>1:
            next_pos = filter_pos_by_minvalue(MESC*-1, next_pos)
            # print("MESV:")
            # print([(MESC[x][y], x, y) for x,y in next_pos])
        if len(next_pos)>1:
            next_pos = filter_pos_by_minvalue(ERV, next_pos)
            # print("ERV:")
            # print([(ERV[x][y], x, y) for x,y in next_pos])
        if len(next_pos)>1:
            next_pos = filter_pos_by_mindist(end, next_pos)

        return next_pos, visited
    
def main(start, end):
    
    visited = add_visited(start, set())
    queue = [(start, [start], visited)]
    path_pool = []

    while queue:
        # print("queue:", queue)
        current_pos, path, visited = queue.pop()
        
        if current_pos == end:
            path_pool.append(path)
            continue
        
        next_pos, visited_new = process_one_round(current_pos, end, visited)
        for cord in next_pos:
            path_new = path.copy() + [cord]
            item = (cord, path_new, visited_new)
            queue.append(item)
            
    return path_pool


In [163]:
ERV = pd.read_csv("/Users/tuanzai/Desktop/Canada/001.UBC/block_capstone/花姐项目/simple_landscape_analysis/history/2.4.4/案例/erv.csv").values
MESC =  pd.read_csv("/Users/tuanzai/Desktop/Canada/001.UBC/block_capstone/花姐项目/simple_landscape_analysis/history/2.4.4/案例/mesc.csv").values

In [164]:
start = [12, 2]
end = [1, 9]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [12, 2]  MESV value: 5  ERV value: 2
Cordination: [11, 2]  MESV value: 5  ERV value: 3
Cordination: [10, 2]  MESV value: 5  ERV value: 4
Cordination: [9, 3]  MESV value: 4  ERV value: 7
Cordination: [8, 3]  MESV value: 5  ERV value: 4
Cordination: [7, 4]  MESV value: 3  ERV value: 5
Cordination: [6, 4]  MESV value: 2  ERV value: 10
Cordination: [5, 5]  MESV value: 4  ERV value: 3
Cordination: [4, 6]  MESV value: 1  ERV value: 4
Cordination: [3, 6]  MESV value: 5  ERV value: 7
Cordination: [2, 6]  MESV value: 0  ERV value: 5
Cordination: [1, 6]  MESV value: 3  ERV value: 4
Cordination: [0, 7]  MESV value: 3  ERV value: 3
Cordination: [1, 8]  MESV value: 1  ERV value: 7
Cordination: [1, 9]  MESV value: 5  ERV value: 3


In [165]:
start = [8, 3]
end = [13, 10]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [8, 3]  MESV value: 5  ERV value: 4
Cordination: [9, 3]  MESV value: 4  ERV value: 7
Cordination: [10, 3]  MESV value: 3  ERV value: 4
Cordination: [11, 3]  MESV value: 2  ERV value: 5
Cordination: [12, 3]  MESV value: 5  ERV value: 3
Cordination: [13, 3]  MESV value: 4  ERV value: 1
Cordination: [14, 4]  MESV value: 1  ERV value: 2
Cordination: [14, 5]  MESV value: 5  ERV value: 4
Cordination: [14, 6]  MESV value: 4  ERV value: 5
Cordination: [14, 7]  MESV value: 5  ERV value: 7
Cordination: [13, 8]  MESV value: 3  ERV value: 5
Cordination: [14, 9]  MESV value: 3  ERV value: 2
Cordination: [13, 10]  MESV value: 5  ERV value: 4


✅ 案例三

In [166]:
start = [3, 6]
end = [12, 5]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [3, 6]  MESV value: 5  ERV value: 7
Cordination: [3, 5]  MESV value: 2  ERV value: 5
Cordination: [4, 4]  MESV value: 2  ERV value: 7
Cordination: [5, 5]  MESV value: 4  ERV value: 3
Cordination: [6, 6]  MESV value: 3  ERV value: 8
Cordination: [7, 6]  MESV value: 1  ERV value: 1
Cordination: [8, 5]  MESV value: 5  ERV value: 7
Cordination: [9, 5]  MESV value: 3  ERV value: 4
Cordination: [10, 5]  MESV value: 5  ERV value: 2
Cordination: [11, 6]  MESV value: 4  ERV value: 2
Cordination: [12, 5]  MESV value: 5  ERV value: 5


案例四、不一样，第五步往下面的2走了

In [167]:
start = [1, 14]
end = [13, 1]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [1, 14]  MESV value: 5  ERV value: 5
Cordination: [2, 13]  MESV value: 5  ERV value: 8
Cordination: [2, 12]  MESV value: 4  ERV value: 5
Cordination: [3, 11]  MESV value: 4  ERV value: 5
Cordination: [4, 11]  MESV value: 4  ERV value: 7
Cordination: [5, 11]  MESV value: 2  ERV value: 10
Cordination: [6, 11]  MESV value: 4  ERV value: 5
Cordination: [7, 10]  MESV value: 2  ERV value: 5
Cordination: [8, 10]  MESV value: 4  ERV value: 5
Cordination: [9, 9]  MESV value: 5  ERV value: 2
Cordination: [10, 8]  MESV value: 4  ERV value: 6
Cordination: [11, 7]  MESV value: 5  ERV value: 1
Cordination: [11, 6]  MESV value: 4  ERV value: 2
Cordination: [12, 5]  MESV value: 5  ERV value: 5
Cordination: [12, 4]  MESV value: 4  ERV value: 4
Cordination: [12, 3]  MESV value: 5  ERV value: 3
Cordination: [12, 2]  MESV value: 5  ERV value: 2
Cordination: [13, 1]  MESV value: 5  ERV value: 5


✅ 案例五

In [169]:
start = [11, 7]
end = [3, 7]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [11, 7]  MESV value: 5  ERV value: 1
Cordination: [10, 8]  MESV value: 4  ERV value: 6
Cordination: [9, 7]  MESV value: 4  ERV value: 1
Cordination: [8, 7]  MESV value: 4  ERV value: 8
Cordination: [7, 8]  MESV value: 4  ERV value: 6
Cordination: [6, 8]  MESV value: 5  ERV value: 10
Cordination: [5, 7]  MESV value: 4  ERV value: 7
Cordination: [4, 8]  MESV value: 2  ERV value: 5
Cordination: [3, 7]  MESV value: 4  ERV value: 1


案例六， 12步之后不一样

In [170]:
start = [0, 13]
end = [13, 13]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [0, 13]  MESV value: 5  ERV value: 5
Cordination: [1, 14]  MESV value: 5  ERV value: 5
Cordination: [2, 13]  MESV value: 5  ERV value: 8
Cordination: [3, 13]  MESV value: 4  ERV value: 6
Cordination: [4, 12]  MESV value: 5  ERV value: 8
Cordination: [5, 12]  MESV value: 3  ERV value: 5
Cordination: [6, 13]  MESV value: 4  ERV value: 5
Cordination: [7, 12]  MESV value: 5  ERV value: 8
Cordination: [8, 12]  MESV value: 2  ERV value: 2
Cordination: [9, 13]  MESV value: 3  ERV value: 9
Cordination: [10, 14]  MESV value: 5  ERV value: 8
Cordination: [11, 14]  MESV value: 1  ERV value: 1
Cordination: [12, 14]  MESV value: 3  ERV value: 8
Cordination: [13, 13]  MESV value: 4  ERV value: 7


✅ 案例七

In [171]:
start = [8, 10]
end = [8, 5]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [8, 10]  MESV value: 4  ERV value: 5
Cordination: [9, 9]  MESV value: 5  ERV value: 2
Cordination: [8, 8]  MESV value: 4  ERV value: 9
Cordination: [9, 7]  MESV value: 4  ERV value: 1
Cordination: [9, 6]  MESV value: 2  ERV value: 2
Cordination: [8, 5]  MESV value: 5  ERV value: 7


✅ 案例八

In [172]:
start = [1, 2]
end = [1, 9]
path_pool = main(start, end)

for path in path_pool:
    print("="*50)
    for i in path:
        print("Cordination:", i, " MESV value:", MESC[i[0], i[1]], " ERV value:", ERV[i[0], i[1]])

Cordination: [1, 2]  MESV value: 5  ERV value: 8
Cordination: [0, 3]  MESV value: 3  ERV value: 4
Cordination: [1, 4]  MESV value: 2  ERV value: 1
Cordination: [1, 5]  MESV value: 2  ERV value: 5
Cordination: [0, 6]  MESV value: 4  ERV value: 1
Cordination: [0, 7]  MESV value: 3  ERV value: 3
Cordination: [1, 8]  MESV value: 1  ERV value: 7
Cordination: [1, 9]  MESV value: 5  ERV value: 3
